<center> <img src="https://images.squarespace-cdn.com/content/v1/588f9607bebafbc786f8c5f8/1607924812500-Y1JR8L6XP5NKF2YPHDUX/image6.png?format=1000w"> </center>

### About Dataset

**Context**
* Predict behavior to retain customers. You can analyze all relevant customer data and develop focused customer retention programs." 

**Content**
* Each row represents a customer, each column contains customer’s attributes described on the column Metadata.

**The data set includes information about:**

* Customers who left within the last month – the column is called Churn
* Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies
* Customer account information – how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges
* Demographic info about customers – gender, age range, and if they have partners and dependents

### <center> Notebook Code starts here </center>

* Importing all the required libraries in this notebook

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy.stats as st
from statsmodels.graphics.gofplots import qqplot
from imblearn.combine import SMOTEENN
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'seaborn'

* It is always considered as a good practice to work on a copy of original dataset.

In [ ]:
main_df = pd.read_csv("CusomterChurn.csv")
df = main_df.copy()
df.head()

* Dimesion of the dataset

In [ ]:
df.shape

* All the columns present in the dataframe

In [ ]:
df.columns

* Basic information about the dataset

In [ ]:
df.info()

* Number of unique input in each columns

In [ ]:
df.nunique()

* Statistical description of the dataset

In [ ]:
df.describe()

* Droppping CustomerID. 

In [ ]:
df = df.drop('customerID', axis=1)
df.head()

* Changing datatype of the column "TotalCharges" because it was initially in object type. 

In [ ]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors="coerce")

* Checking for any null value in the dataset.

In [ ]:
df.isnull().sum()

* Dropping the rows having missing value in the dataset

In [3]:
df.drop(df[df['TotalCharges'].isnull()].index, inplace=True)
df.reset_index(drop=True, inplace=True)

NameError: name 'df' is not defined

In [ ]:
df.head()

In [ ]:
df.shape

* Visualizing for the null value.

In [ ]:
sns.heatmap(df.isnull())

* checking the correlation values of the variables.

In [ ]:
sns.heatmap(df.corr(), annot=True)

* Checking skew value of the variable
* Here SeniorCitizen is highly right skew
* Here TotalCharges is moderately right skew

In [ ]:
skew_val = df.skew().sort_values(ascending=False)
skew_val

### Uni-variate Analysis

In [ ]:
sns.displot(x="TotalCharges", data=df, kde=True)
description = df['TotalCharges'].describe()
plt.axvline(description["25%"], ls="--", color='r')
plt.axvline(description["mean"], ls="--", color='r')
plt.axvline(description["75%"], ls="--", color='r')

* Above plot shows us the data is highly right skew.

In [ ]:
df['TotalCharges'].skew()

In [ ]:
fig, (ax1,ax2) = plt.subplots(ncols=2, figsize=(15,4))
sns.distplot(df['TotalCharges'], ax=ax1 , color ='red')
ax1.set(title='TotalCharges distribution')
qqplot(df['TotalCharges'], ax=ax2, line='s')
ax2.set(title='Quantile quantile plot')

In [ ]:
# MonthlyCharges

In [ ]:
sns.displot(x="MonthlyCharges", data=df, kde=True)
description = df['MonthlyCharges'].describe()
plt.axvline(description["25%"], ls="--", color='r')
plt.axvline(description["mean"], ls="--", color='r')
plt.axvline(description["75%"], ls="--", color='r')

In [ ]:
# df['MonthlyCharges'].skew()

In [ ]:
fig, (ax1,ax2) = plt.subplots(ncols=2, figsize=(15,4))
sns.distplot(df['MonthlyCharges'], ax=ax1 , color ='red')
ax1.set(title='MonthlyCharges distribution')
qqplot(df['MonthlyCharges'], ax=ax2, line='s')
ax2.set(title='Quantile quantile plot')

In [ ]:
# for i, predictor in enumerate(df.columns):
#     plt.figure(i)
#     sns.countplot(data=df, x=predictor, hue='Churn')

In [ ]:
# Bi-variate analysis

In [ ]:
plt.figure(figsize=(5, 5))
sns.barplot(data = df, y="TotalCharges", x="Churn")

In [ ]:
px.scatter(df, y="TotalCharges", x="tenure")

In [ ]:
diag = px.histogram(df, x="Churn", color="SeniorCitizen")
# diag.update_layout(width=750, height=550)
diag.show()

In [ ]:
df.info()

In [ ]:
diag = px.histogram(df, x="Churn", color="gender")
diag.show()

In [ ]:
diag = px.histogram(df, x="Churn", color="PhoneService")
diag.show()

In [ ]:
diag = px.pie(df, values='TotalCharges', names='Churn', hole=0.5)
diag.show()

In [ ]:
labels = df['MultipleLines'].unique()
values = df['MultipleLines'].value_counts()

# pull is given as a fraction of the pie radius
diag = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0.1, 0.2])])
diag.show()

In [ ]:
labels = df['InternetService'].unique()
values = df['InternetService'].value_counts()


diag = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0.2, 0.3])])
diag.show()

In [ ]:
labels = df['PaymentMethod'].unique()
values = df['PaymentMethod'].value_counts()


diag = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0, 0.2, 0])])
diag.show()

In [ ]:
labels = df['Contract'].unique()
values = df['Contract'].value_counts()


diag = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0.2, 0.3])])
diag.show()

In [ ]:
print (df['Partner'].value_counts(ascending=True))

In [ ]:
for i in df.columns:
    if df[i].dtypes=="object":
        print(f'{i} : {df[i].unique()}')
        print("****************************************************")

In [ ]:
df.replace('No internet service', 'No', inplace=True)
df.replace('No phone service', 'No', inplace=True)

In [ ]:
for i in df.columns:
    if df[i].dtypes=="object":
        print(f'{i} : {df[i].unique()}')
        print("****************************************************")

In [ ]:
print(df['gender'].value_counts(ascending=True))

In [ ]:
print(df['Churn'].value_counts(ascending=True))

In [ ]:
df['gender'].replace({'Female':1,'Male':0},inplace=True)
df.head()

In [ ]:
print(df['InternetService'].value_counts(ascending=True))

In [ ]:
for i in df.columns:
    if (len(df[i].unique()) >2) & (df[i].dtypes != "int64") &(df[i].dtypes!= "float64"):
        print(i)

In [ ]:
print(df['InternetService'].value_counts(ascending=True))

In [ ]:
print(df['Contract'].value_counts(ascending=True))

In [ ]:
print(df['PaymentMethod'].value_counts(ascending=True))

In [ ]:
more_than_2 = ['InternetService' ,'Contract' ,'PaymentMethod']
df = pd.get_dummies(data=df, columns= more_than_2,drop_first=True )
df.dtypes

In [ ]:
# df.drop(['DSL', 'Month-to-month'], axis = 1)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
for i in df.columns:
    if (df[i].dtypes == "int64")  | (df[i].dtypes== "float64"):
        print(i)

In [ ]:
df.head()

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()

In [ ]:
# large_cols = ["tenure", "MonthlyCharges", "TotalCharges"]
# df[large_cols] = scaler.fit_transform(df[large_cols])
# df[large_cols].head()

In [ ]:
# After feature scaling we have following dataset

df.head()

In [ ]:
for i in df.columns:
    if (df[i].dtypes == "object"):
        print(i)

In [ ]:
two_cate = ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn']
for i in two_cate:
    df[i].replace({"No":0, "Yes":1}, inplace=True)
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
X = df.drop('Churn', axis=1)
y = df['Churn']

In [ ]:
X.shape, y.shape

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
oversample = SMOTE()

In [ ]:
X1, y1 = oversample.fit_resample(X,y)

In [ ]:
X1.shape, y1.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X1, y1, test_size=0.33, random_state=0)

In [ ]:
model_rf=RandomForestClassifier(criterion = "gini",random_state = 10,max_depth=10, min_samples_leaf=5)

In [ ]:
model_rf.fit(X_train,y_train)

In [ ]:
pred_rf = model_rf.predict(X_test)

In [ ]:
rf  = round(accuracy_score(y_test, pred_rf)*100, 2)
print(rf)    

In [ ]:
pred=model_rf.predict(X_train)
print(accuracy_score(pred,y_train))

In [ ]:
print(classification_report(y_test, pred_rf))

In [ ]:
# confusion Maxtrix
cm = confusion_matrix(y_test, pred_rf)
sns.heatmap(cm/np.sum(cm), annot = True, fmt=  '0.2%', cmap = 'Reds')
plt.title("Random Forest Confusion Matrix",fontsize=12)
plt.show()

In [ ]:
filename = 'model.sav'

In [ ]:
pickle.dump(model_rf, open(filename, 'wb'))

In [ ]:
load_model = pickle.load(open(filename, 'rb'))

In [ ]:
model_score = load_model.score(X_test, y_test)

In [ ]:
model_score

## Prediction

In [ ]:
gender = ['Female', 'Male']
Partner = ['Yes','No']
Dependents = ['Yes','No']
PhoneService = ['Yes','No']
MultipleLines = ['Yes','No']
OnlineBackup = ['Yes','No']
DeviceProtection = ['Yes','No']
InternetService = ['No', 'DSL','Fiber optic']
OnlineSecurity = ['Yes','No']
TechSupport  = ['Yes','No']
StreamingTV  = ['Yes','No']
StreamingMovies  = ['Yes','No']
Contract =  ['Month-to-month', 'One year' ,'Two year']
PaperlessBilling = ['Yes','No']
PaymentMethod  =  ['Bank transfer (automatic)', 'Credit card (automatic)' , 'Mailed check', 'Electronic check']

In [ ]:
gender = pd.DataFrame(gender)
Partner = pd.DataFrame(Partner)
Dependents =  pd.DataFrame(Dependents)
PhoneService =  pd.DataFrame(PhoneService)
MultipleLines =  pd.DataFrame(MultipleLines)
OnlineBackup = pd.DataFrame(OnlineBackup)
DeviceProtection =  pd.DataFrame(DeviceProtection)
InternetService =  pd.DataFrame(InternetService)
OnlineSecurity =  pd.DataFrame(OnlineSecurity)
TechSupport  = pd.DataFrame(TechSupport)
StreamingTV  = pd.DataFrame(StreamingTV)
StreamingMovies  =  pd.DataFrame(StreamingMovies)
Contract =   pd.DataFrame(Contract)
PaperlessBilling =  pd.DataFrame(PaperlessBilling)
PaymentMethod  =   pd.DataFrame(PaymentMethod)

In [ ]:
# SeniorCitizen = [0 , 1]

In [ ]:
# Prediction

In [ ]:
data = pd.concat([gender,Partner,Dependents,PhoneService , MultipleLines, OnlineBackup,DeviceProtection, InternetService,OnlineSecurity ,TechSupport, StreamingTV, StreamingMovies, Contract, PaperlessBilling, PaymentMethod],axis=1)
data

In [ ]:
data.columns=['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineBackup', 'DeviceProtection', 'InternetService', 'OnlineSecurity','TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']
data.head()

In [ ]:
data.info()

In [4]:
# data.isnull().sum()

data = data.fillna(0)

NameError: name 'data' is not defined

In [ ]:
data.isnull().sum()

In [ ]:
two_cat = ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']
for i in two_cat:
    data[i].replace({"No":0, np.nan:0, "Yes":1}, inplace=True)
data.head()

In [ ]:
data['gender'].replace({"Male":0, "Female":1}, inplace=True)
data

In [ ]:
data.info()

In [ ]:
list(data.select_dtypes(include=['object']).columns)

In [ ]:
more_than2_val =  ['InternetService' ,'Contract' ,'PaymentMethod']
data = pd.get_dummies(data=data, columns = more_than2_val)
data.dtypes


In [ ]:
data = data.drop(columns=['InternetService_DSL'], axis=1)
data = data.drop(columns=['Contract_Month-to-month'], axis=1)
data = data.drop(columns=['PaymentMethod_Bank transfer (automatic)'], axis=1)

In [ ]:
# data

In [ ]:
# categorical_cols = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineBackup', 'DeviceProtection', 'InternetService', 'OnlineSecurity','TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']
# data = pd.get_dummies(data, columns = categorical_cols, drop_first = True )
# data.head()

In [ ]:
data.info()

In [ ]:
dat=data.head(1)
dat

In [ ]:
for col in dat.columns:
    dat[col].values[:] = 0
dat

In [ ]:
tenure = int(input('Enter tenure : '))
MonthlyCharges = float(input('Enter MonthlyCharges : '))
TotalCharges = float(input('Enter TotalCharges : '))
SeniorCitizen = int(input('Enter SeniorCitizen type : '))

gender = input('Enter gender Male or Female : ')
Partner = input('Enter Partner type : ')
Dependents =  input('Enter Dependents type : ')
PhoneService = input('Enter PhoneService type : ')
MultipleLines = input('Enter MultipleLines type : ')
OnlineBackup =  input('Enter OnlineBackup type : ')
DeviceProtection =   input('Enter DeviceProtection type : ')
OnlineSecurity =  input('Enter OnlineSecurity type : ') 
TechSupport  = input('Enter TechSupport type : ')
StreamingTV  = input('Enter StreamingTV type : ')
StreamingMovies  =   input('Enter StreamingMovies type : ')
PaperlessBilling =  input('Enter PaperlessBilling type : ')

In [ ]:
InternetService =  input('Enter "DSL" or "Fiber optic" or "No"  : ')
Contract =    input('Enter "Month-to-month" or "One year" or "Two year"  : ')
PaymentMethod  =  input('Enter "Electronic check" or "Mailed check" or "Credit card (automatic)"  or "Bank transfer (automatic)" : ')

In [ ]:
# data

In [ ]:
# more_than_2_again = ['InternetService' ,'Contract' ,'PaymentMethod']
# data = pd.get_dummies(data, columns = more_than_2_again, drop_first = True )
# data.head()

In [ ]:
new_number=[tenure,MonthlyCharges,TotalCharges,SeniorCitizen,gender,Partner,Dependents,PhoneService , MultipleLines, OnlineBackup,DeviceProtection, InternetService,OnlineSecurity ,TechSupport, StreamingTV, StreamingMovies, Contract, PaperlessBilling, PaymentMethod]
data_new = pd.DataFrame(new_number)
data_new=data_new.T

data_new.columns = ['tenure','MonthlyCharges','TotalCharges','SeniorCitizen','gender','Partner','Dependents','PhoneService' , 'MultipleLines', 'OnlineBackup','DeviceProtection', 'InternetService','OnlineSecurity' ,'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']
data_new

In [ ]:
data_new.shape

In [ ]:
# new_number=[tenure,MonthlyCharges,TotalCharges,SeniorCitizen,gender,Partner,Dependents,PhoneService , MultipleLines, OnlineBackup,DeviceProtection, InternetService,OnlineSecurity ,TechSupport, StreamingTV, StreamingMovies, Contract, PaperlessBilling, PaymentMethod]
# data_new = pd.DataFrame(new_number)
# data_new=data_new.T
# data_new

In [ ]:
# data_new['gender'][0]

In [ ]:
data_new.columns = ['tenure','MonthlyCharges','TotalCharges','SeniorCitizen','gender', "Partner",  'Dependents', 'PhoneService', 'MultipleLines', 'OnlineBackup', 'DeviceProtection', f"InternetService_{data_new['InternetService'][0]}",  'OnlineSecurity', 'TechSupport',  'StreamingTV',  'StreamingMovies',  f"Contract_{data_new['Contract'][0]}", 'PaperlessBilling', f"PaymentMethod_{data_new['PaymentMethod'][0]}"]
data_new

In [ ]:
data_new.shape

In [ ]:
num=data_new.iloc[:,0:4]
cat=data_new.iloc[:,4:]

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# scale = MinMaxScaler()

In [ ]:
# large_num = ["tenure", "MonthlyCharges", "TotalCharges"]
# num[large_num] = scale.fit_transform(num[large_num])
# num[large_num]

In [ ]:
num

In [ ]:
cat

In [ ]:
lis=[]
for i in cat.columns:
    if i.endswith("DSL") or i.endswith('Month-to-month') or i.endswith('Bank transfer (automatic)') :
        print("true ")
        lis.append(i)
    else:
        print("false")
        
print(lis)

In [ ]:
cat=cat.drop(columns=lis, axis=1)

In [ ]:
cat

In [ ]:
for col in cat.columns:
    dat[col].values[:] = 1

In [ ]:
dat.info()

In [ ]:
new=pd.concat([num,dat], axis=1)

In [ ]:
new

In [ ]:
new.info()

In [ ]:
new.shape

In [ ]:
li=[]
for i in new.columns:
    if i.endswith("0"):
        print("true ")
        li.append(i)
    else:
        print("false")
        
print(li)

In [ ]:
new=new.drop(columns=li, axis=1)
# new.info()

In [ ]:
print(model_rf.predict(new))